In [ ]:
#%%appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [ ]:
import json
import requests
from time import sleep
from tabulate import tabulate
from IPython.display import HTML, display

In [ ]:
def get_kea3_results(gene_set, query_name):
    ADDLIST_URL = 'https://amp.pharm.mssm.edu/kea3/api/enrich/'
    payload = {
        'gene_set': gene_set,
        'query_name': query_name
    }

    response = requests.post(ADDLIST_URL, data=json.dumps(payload))
    if not response.ok:
        raise Exception('Error analyzing gene list')
    sleep(1)

    return json.loads(response.text)

In [ ]:
%%appyter hide_code

{% do SectionField(name ='section', title = 'Perform Kinase Enrichment Analysis (KEA) 3', 
                   subtitle = 'Kinase Enrichment Analysis 3 (KEA3) infers upstream kinases whose putative substrates are ' +
                               'overrepresented in a user-inputted list of genes or differentially phosphorylated proteins. ' +
                               'The KEA3 database contains putative kinase-substrate interactions collected from publicly ' +
                               'available datasets. Upload or input a gene/protein list to retrieve the top associated kinases.', 
                   img = 'poll.png')%}

In [ ]:
%%appyter markdown

### Generate table containing top associated kinases for different libraries 
### (supersets aggregarted based on database from which they were derived)

In [ ]:
%%appyter code_exec

gene_list_file = {{ FileField(
        name = 'Upload gene/protein list',
        label = 'Upload gene/protein list',
        default = '',
        section = 'section'
    )}}

            
gene_list_input = {{ TextField(
        name = 'Input gene/protein list',
        label = 'Input gene/protein list (one per row)',
        default = 'ZNF264 \n TMPO \n ISL2 \n MAP3K8 \n EFNB1 \n EIF3C \n OSBPL11 \n ABCF1 \n UTRN \n OPRK1 \n TSC1 ' +
                    '\n GAB2 \n RPS3P2 \n DDX3X \n PPP1CA \n NF2 \n RBM3 \n IRAK1 \n KCNH2 \n NPR1 \n MOCOS \n ITSN2 ' +
                    '\n MITF \n ARAF \n DAPK2 \n EPHB2 \n CACNA1G \n YWHAZ \n GMFB',
        section = 'section',
    )}}

# Number of ranked kinases to display
display_num_kinases = {{ IntField(
        name = 'Input number of top kinases to display',
        label = 'Input number of top kinases to display',
        default = '10',
        section = 'section'
    )}}

input_exists = False 

if gene_list_file != '':
    open_gene_list_file = open(gene_list_file,'r')
    lines = open_gene_list_file.readlines()
    genes = [x.strip() for x in lines]
    open_gene_list_file.close()
    input_exists = True 
elif gene_list_input != '':
    genes = gene_list_input.split('\n')
    genes = [x.strip() for x in genes]
    input_exists = True 
else:
    display(HTML('<h2>No kinase list was inputted or uploaded.</h2>'))

if input_exists:
    results = get_kea3_results(genes, 'Query')

    # Create dictionary of dictionaries for the top 10 kinases for each ranking type
    # Format: rankings['Rank Type'] = {1: Kinase, 2: Kinase, 3: Kinase, ...}
    rankings = {}
    for library in range(0, 13):
        rankings[list(results.keys())[library]] = {}
        for rank in range(1, 11):
            rankings[list(results.keys())[library]].update({rank: list(results.values())[library][rank-1]['TF']})

    # Create list of lists
    # Format: ranks[0] will contain all of the #1 kinases for each ranking type 
    ranks = []
    for rank_num in range(0, 11):
        ranks.append([rank_num+1])

    for library in range(0, 13):
        for rank in range(1, 11):
            ranks[rank-1].append(list(rankings[list(results.keys())[library]].values())[rank-1])

    display(HTML(tabulate([ranks[0], ranks[1], ranks[2], ranks[3], ranks[4], ranks[5], ranks[6], 
                           ranks[7], ranks[8], ranks[9]], 
                          headers = [list(rankings.keys())[0], list(rankings.keys())[1], list(rankings.keys())[2], 
                                     list(rankings.keys())[3], list(rankings.keys())[4], list(rankings.keys())[5], 
                                     list(rankings.keys())[6], list(rankings.keys())[7], list(rankings.keys())[8], 
                                     list(rankings.keys())[9], list(rankings.keys())[10], list(rankings.keys())[11],
                                     list(rankings.keys())[12]],
                          tablefmt = 'html' )))
